# Imports

In [1]:
import os
import json
import PyPDF2
import boto3
import nltk
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from tensorflow import keras
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer


# Embeddings Qdrant

In [2]:
# %% [markdown]
# ## Paso 2: Configuración de Qdrant, Amazon Bedrock y variable de modelos
# Ajusta el host/puerto de Qdrant y el ID del modelo de Amazon Bedrock según tu configuración.

# Inicializamos el cliente de Qdrant
qdrant = QdrantClient(host="localhost", port=6333)  # Cambia estos parámetros según tu entorno
collection_name = "documentos"

# Variable de modelos (ejemplo); se debe configurar con el id del modelo en Amazon Bedrock
modelos = {
    "Titan Embeddings G1 - Text" : "amazon.titan-embed-text-v1",
    "Titan Text G1 - Lite": "amazon.titan-text-lite-v1",
    "Titan Text G1 - Express": "amazon.titan-text-express-v1",
    "": "",
    "Rerank_1.0": "amazon.rerank-v1:0",
    "Claude_3.5_Sonnet" : "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "Claude_3_Sonnet": "anthropic.claude-3-sonnet-20240229-v1:0",
    "Claude_3_Haiku" : "anthropic.claude-3-5-haiku-20241022-v1:0",
    "Claude_2.1" : "anthropic.claude-v2:1",
    "Claude_Instant" : "anthropic.claude-instant-v1",
    "Claude": "anthropic.claude-v2:0"
}

# Inicializamos el cliente de Amazon Bedrock
bedrock = boto3.client('bedrock-runtime')

In [3]:
# %% [markdown]
# ## Paso 3: Función para leer el PDF y extraer el texto

def read_pdf(file_path):
    """Lee un PDF y extrae todo su texto."""
    text = ""
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# Ruta al documento
pdf_path = "documento.pdf"
print("Leyendo el PDF...")
document_text = read_pdf(pdf_path)
print("PDF leído correctamente.")

Leyendo el PDF...
PDF leído correctamente.


In [4]:
# %% [markdown]
# ## Paso 4: Función para dividir el texto en chunks
# Se define un tamaño de chunk y un solapamiento para conservar contexto entre ellos.

def chunk_text(text, chunk_size=1000, overlap=100):
    """
    Divide el texto en chunks de `chunk_size` caracteres con un solapamiento de `overlap` caracteres.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += (chunk_size - overlap)
    return chunks

print("Dividiendo el documento en chunks...")
chunks = chunk_text(document_text, chunk_size=1000, overlap=100)
print(f"Total de chunks generados: {len(chunks)}")

Dividiendo el documento en chunks...
Total de chunks generados: 478


In [5]:
# %% [markdown]
# ## Paso 5: Calcular embeddings y subir a Qdrant
# Se usa un modelo de SentenceTransformer para obtener los embeddings de cada chunk.
# Si la colección en Qdrant no existe, se crea con la dimensión correcta.

# Inicializamos el modelo de embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Obtenemos la dimensión del embedding
dimension = embedder.get_sentence_embedding_dimension()

# Creamos la colección en Qdrant si no existe
collections = qdrant.get_collections().collections
if not any(col.name == collection_name for col in collections):
    print("Creando colección en Qdrant...")
    qdrant.create_collection(
         collection_name=collection_name,
         vectors_config=VectorParams(size=dimension, distance="Cosine")
    )
    print("Colección creada.")

print("Computando embeddings de los chunks...")
embeddings = embedder.encode(chunks, show_progress_bar=True)

# Preparamos los puntos (cada punto asocia un chunk y su embedding)
points = []
for i, (chunk, vector) in enumerate(zip(chunks, embeddings)):
    points.append(PointStruct(
         id=i,
         vector=vector.tolist(),
         payload={"text": chunk}
    ))

print("Subiendo los embeddings a Qdrant...")
qdrant.upsert(collection_name=collection_name, points=points)
print("Embeddings subidos correctamente.")

Computando embeddings de los chunks...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Subiendo los embeddings a Qdrant...
Embeddings subidos correctamente.


In [6]:
# %% [markdown]
# ## Paso 6: Función para recuperar chunks relevantes desde Qdrant
# Dada una query, se calcula su embedding y se buscan los chunks más similares.

def get_relevant_chunks(query, top_k=5):
    """
    Dada una query, recupera los `top_k` chunks más relevantes desde Qdrant.
    """
    query_embedding = embedder.encode(query)
    search_result = qdrant.search(
         collection_name=collection_name,
         query_vector=query_embedding.tolist(),
         limit=top_k
    )
    # Extraemos el texto de cada chunk recuperado
    contexts = [res.payload.get("text", "") for res in search_result]
    return contexts

# Multi-agent System

In [7]:
from qdrant_client import QdrantClient
# Integración de Amazon Bedrock via LangChain AWS
from langchain_aws import BedrockLLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.chains import RetrievalQA

In [10]:
# ---------------------------------------------------------
# Configuración
# ---------------------------------------------------------
QDRANT_HOST = os.getenv("QDRANT_HOST", "localhost")
QDRANT_PORT = int(os.getenv("QDRANT_PORT", 6333))
COLLECTION_NAME = "documentos"
# Perfil AWS para Bedrock (configurar en ~/.aws/credentials)
BEDROCK_PROFILE = os.getenv("BEDROCK_PROFILE", "bedrock-admin")
# ID del modelo Bedrock a usar
BEDROCK_MODEL_ID = os.getenv("BEDROCK_MODEL_ID", "amazon.titan-text-express-v1")

# ---------------------------------------------------------
# Inicialización
# ---------------------------------------------------------
# Cliente Qdrant
# qdrant_client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
# 
# # Embeddings con HuggingFace
# embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# VectorStore basado en Qdrant
vectorstore = QdrantVectorStore(
    client=qdrant,
    collection_name=collection_name,
    embedding=embedder
)

# Recuperador de los top_k chunks relevantes
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# LLM de Amazon Bedrock
llm = BedrockLLM(
    credentials_profile_name="Claude_3.5_Sonnet",
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
)

# Cadena RAG: recuperación + generación
qa_agent = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=False
)

# ---------------------------------------------------------
# Función principal de procesamiento
# ---------------------------------------------------------
def process_questions(input_json: str, output_json: str):
    """
    Carga preguntas de `input_json`, consulta al agente Bedrock QA,
    y salva en `output_json` con question/expected/generated.
    """
    with open(input_json, 'r', encoding='utf-8') as f:
        qa_list = json.load(f)

    results = []
    for item in qa_list:
        question = item.get("Q") or item.get("question")
        expected = item.get("A") or item.get("expected_answer")

        # Invocar la cadena QA
        output = qa_agent.invoke({"query": question})
        generated = output.get("result")

        results.append({
            "question": question,
            "expected_answer": expected,
            "generated_answer": generated.strip() if isinstance(generated, str) else generated
        })

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f"Resultados guardados en {output_json}")

# ---------------------------------------------------------
# Ejecución si se lanza como script
# ---------------------------------------------------------
if __name__ == "__main__":
    process_questions(
        input_json="Expert-questions.json",
        output_json="Expert-questions-output.json"
    )
    process_questions(
        input_json="Not-expert-questions.json",
        output_json="Not-expert-questions-output.json"
    )


ValueError: Invalid `embeddings` type.